In [39]:
import os
import requests

import earthaccess
import pystac_client
import rioxarray
import shapely

/opt/miniconda3/envs/modis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ['GDAL_HTTP_COOKIEFILE'] = '~/cookies.txt'
os.environ['GDAL_HTTP_COOKIEJAR'] = '~/cookies.txt'

# Access MODIS data from a workstation

Authentication is carried out via [NASA Earthdata Login (EDL](https://urs.earthdata.nasa.gov), credentials are stored in the `~/.netrc` file.

## Search data

In [3]:
stac_api = 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD'
collection = 'MOD13Q1.v061'
conus_bbox = (-124.784, 24.743, -66.951, 49.346)
datetime_range = '2019-01-01/2019-01-31'

In [4]:
client = pystac_client.Client.open(stac_api)

In [5]:
search = client.search(
    collections=[collection],
    bbox=conus_bbox,
    datetime=datetime_range,
)

In [6]:
search.matched()

51

In [7]:
item_collection = search.item_collection()

In [8]:
href = item_collection[0].assets['data'].href
href

'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD13Q1.061/MOD13Q1.A2018353.h10v06.061.2021361143553/MOD13Q1.A2018353.h10v06.061.2021361143553.hdf'

## Access data directly

It fails, probably becaused of the format (HDF4):

In [35]:
ds = rioxarray.open_rasterio(href)

RasterioIOError: '/vsicurl/https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD13Q1.061/MOD13Q1.A2018353.h10v06.061.2021361143553/MOD13Q1.A2018353.h10v06.061.2021361143553.hdf' not recognized as a supported file format.

In [36]:
!gdalinfo $href

Driver: HDF4/Hierarchical Data Format Release 4
Files: none associated
Size is 512, 512
Metadata:
  ALGORITHMPACKAGEACCEPTANCEDATE=102004
  ALGORITHMPACKAGEMATURITYCODE=Normal
  ALGORITHMPACKAGENAME=MOD_PR13A1
  ALGORITHMPACKAGEVERSION=6
  ASSOCIATEDINSTRUMENTSHORTNAME.1=MODIS
  ASSOCIATEDPLATFORMSHORTNAME.1=Terra
  ASSOCIATEDSENSORSHORTNAME.1=MODIS
  AUTOMATICQUALITYFLAG.1=Passed
  AUTOMATICQUALITYFLAG.10=Passed
  AUTOMATICQUALITYFLAG.11=Passed
  AUTOMATICQUALITYFLAG.12=Passed
  AUTOMATICQUALITYFLAG.2=Passed
  AUTOMATICQUALITYFLAG.3=Passed
  AUTOMATICQUALITYFLAG.4=Passed
  AUTOMATICQUALITYFLAG.5=Passed
  AUTOMATICQUALITYFLAG.6=Passed
  AUTOMATICQUALITYFLAG.7=Passed
  AUTOMATICQUALITYFLAG.8=Passed
  AUTOMATICQUALITYFLAG.9=Passed
  AUTOMATICQUALITYFLAGEXPLANATION.1=No automatic quality assessment is performed in the PGE
  AUTOMATICQUALITYFLAGEXPLANATION.10=No automatic quality assessment is performed in the PGE
  AUTOMATICQUALITYFLAGEXPLANATION.11=No automatic quality assessment is perf

Testing for HLS, for which data is stored in COG format:

In [11]:
search_tmp = client.search(
    collections=['HLSS30.v2.0'],
    intersects=shapely.Point(-90, 35),
    datetime=datetime_range,
)

In [12]:
search_tmp.matched()

9

In [13]:
item_collection_tmp = search_tmp.item_collection()

In [14]:
href_tmp = item_collection_tmp[0].assets['B02'].href
href_tmp

'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T16SBD.2019006T164659.v2.0/HLS.S30.T16SBD.2019006T164659.v2.0.B02.tif'

In [15]:
da = rioxarray.open_rasterio(href_tmp)

In [16]:
da

<xarray.DataArray (band: 1, y: 3660, x: 3660)>
[13395600 values with dtype=int16]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 2e+05 2e+05 2.001e+05 ... 3.097e+05 3.098e+05
  * y            (y) float64 3.9e+06 3.9e+06 3.9e+06 ... 3.79e+06 3.79e+06
    spatial_ref  int64 0
Attributes: (12/42)
    ACCODE:                                            LaSRC
    add_offset:                                        0.0
    AREA_OR_POINT:                                     Area
    arop_ave_xshift(meters):                           0
    arop_ave_yshift(meters):                           0
    arop_ncp:                                          0
    ...                                                ...
    spatial_coverage:                                  99
    SPATIAL_RESOLUTION:                                30
    TILE_ID:                                           S2B_OPER_MSI_L1C_TL_EP...
    ULX:                                               199980
    ULY:                                               3900000
    _FillValue:                                        -9999

## Download data before reading

Downloading before reading works!

In [29]:
def download(url):
    get_response = requests.get(url,stream=True)
    file_name  = url.split('/')[-1]
    with open(file_name, 'wb') as f:
        for chunk in get_response.iter_content(chunk_size=1024):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    return file_name

In [30]:
file_name = download(href)

In [31]:
!ls $file_name

MOD13Q1.A2018353.h10v06.061.2021361143553.hdf


In [34]:
!gdalinfo $file_name

Driver: HDF4/Hierarchical Data Format Release 4
Files: MOD13Q1.A2018353.h10v06.061.2021361143553.hdf
Size is 512, 512
Metadata:
  ALGORITHMPACKAGEACCEPTANCEDATE=102004
  ALGORITHMPACKAGEMATURITYCODE=Normal
  ALGORITHMPACKAGENAME=MOD_PR13A1
  ALGORITHMPACKAGEVERSION=6
  ASSOCIATEDINSTRUMENTSHORTNAME.1=MODIS
  ASSOCIATEDPLATFORMSHORTNAME.1=Terra
  ASSOCIATEDSENSORSHORTNAME.1=MODIS
  AUTOMATICQUALITYFLAG.1=Passed
  AUTOMATICQUALITYFLAG.10=Passed
  AUTOMATICQUALITYFLAG.11=Passed
  AUTOMATICQUALITYFLAG.12=Passed
  AUTOMATICQUALITYFLAG.2=Passed
  AUTOMATICQUALITYFLAG.3=Passed
  AUTOMATICQUALITYFLAG.4=Passed
  AUTOMATICQUALITYFLAG.5=Passed
  AUTOMATICQUALITYFLAG.6=Passed
  AUTOMATICQUALITYFLAG.7=Passed
  AUTOMATICQUALITYFLAG.8=Passed
  AUTOMATICQUALITYFLAG.9=Passed
  AUTOMATICQUALITYFLAGEXPLANATION.1=No automatic quality assessment is performed in the PGE
  AUTOMATICQUALITYFLAGEXPLANATION.10=No automatic quality assessment is performed in the PGE
  AUTOMATICQUALITYFLAGEXPLANATION.11=No automa

In [32]:
ds = rioxarray.open_rasterio(file_name)

In [33]:
ds

<xarray.Dataset>
Dimensions:                                 (band: 1, x: 4800, y: 4800)
Coordinates:
  * band                                    (band) int64 1
  * x                                       (x) float64 -8.895e+06 ... -7.784...
  * y                                       (y) float64 3.336e+06 ... 2.224e+06
    spatial_ref                             int64 0
Data variables:
    250m 16 days NDVI                       (band, y, x) int16 ...
    250m 16 days relative azimuth angle     (band, y, x) int16 ...
    250m 16 days composite day of the year  (band, y, x) int16 ...
    250m 16 days pixel reliability          (band, y, x) int8 ...
    250m 16 days EVI                        (band, y, x) int16 ...
    250m 16 days VI Quality                 (band, y, x) uint16 ...
    250m 16 days red reflectance            (band, y, x) int16 ...
    250m 16 days NIR reflectance            (band, y, x) int16 ...
    250m 16 days blue reflectance           (band, y, x) int16 ...
    250m 16 days MIR reflectance            (band, y, x) int16 ...
    250m 16 days view zenith angle          (band, y, x) int16 ...
    250m 16 days sun zenith angle           (band, y, x) int16 ...
Attributes: (12/173)
    ALGORITHMPACKAGEACCEPTANCEDATE:      102004
    ALGORITHMPACKAGEMATURITYCODE:        Normal
    ALGORITHMPACKAGENAME:                MOD_PR13A1
    ALGORITHMPACKAGEVERSION:             6
    ASSOCIATEDINSTRUMENTSHORTNAME.1:     MODIS
    ASSOCIATEDPLATFORMSHORTNAME.1:       Terra
    ...                                  ...
    SOUTHBOUNDINGCOORDINATE:             19.9999999982039
    SPSOPARAMETERS:                      2749, 4334, 2749a, 4334a
    TileID:                              51010006
    VERSIONID:                           61
    VERTICALTILENUMBER:                  6
    WESTBOUNDINGCOORDINATE:              -92.3760430595367

## Access via `earthaccess` / `fsspec`

In [40]:
auth = earthaccess.login()

EARTHDATA_USERNAME and EARTHDATA_PASSWORD are not set in the current environment, try setting them or use a different strategy (netrc, interactive)
You're now authenticated with NASA Earthdata Login
Using token with expiration date: 10/22/2023
Using .netrc file for EDL


In [42]:
session = earthaccess.get_fsspec_https_session()

Same issues with the `fsspec` reader setup via `earthaccess`:

In [44]:
with session.open(href) as f:
    ds = rioxarray.open_rasterio(f)

RasterioIOError: '/vsipythonfilelike/02121cbd-eb57-4912-92a8-5f03c6417447/02121cbd-eb57-4912-92a8-5f03c6417447' not recognized as a supported file format.

Work with COGs:

In [48]:
with session.open(href_tmp) as f:
    da = rioxarray.open_rasterio(f)

In [49]:
da

<xarray.DataArray (band: 1, y: 3660, x: 3660)>
[13395600 values with dtype=int16]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 2e+05 2e+05 2.001e+05 ... 3.097e+05 3.098e+05
  * y            (y) float64 3.9e+06 3.9e+06 3.9e+06 ... 3.79e+06 3.79e+06
    spatial_ref  int64 0
Attributes: (12/42)
    ACCODE:                                            LaSRC
    add_offset:                                        0.0
    AREA_OR_POINT:                                     Area
    arop_ave_xshift(meters):                           0
    arop_ave_yshift(meters):                           0
    arop_ncp:                                          0
    ...                                                ...
    spatial_coverage:                                  99
    SPATIAL_RESOLUTION:                                30
    TILE_ID:                                           S2B_OPER_MSI_L1C_TL_EP...
    ULX:                                               199980
    ULY:                                               3900000
    _FillValue:                                        -9999